In [1]:
from import_src import import_src
import_src()

Append /home/ubuntu/khiempm/MLOpsVN/src to sys.path


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import json
from plotly import express as px
import mlflow

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
os.environ.get("MLFLOW_TRACKING_URI")

'http://localhost:5000'

In [4]:
# mlflow_evn = os.environ.get("MLFLOW_TRACKING_URI")
mlflow_evn = "http://localhost:5000"

In [49]:
mlflow.set_tracking_uri(mlflow_evn)
model_uri = os.path.join(
    "models:/", "phase-2_prob-2_lgbm__add_captured_data", "2"
)
input_schema = mlflow.models.Model.load(model_uri).get_input_schema().to_dict()
model = mlflow.sklearn.load_model(model_uri)
get_features = [each['name'] for each in input_schema]

In [5]:
link_data_test = '../../data/captured_data/phase-2/prob-2/'
uncertain_data = '../../data/captured_data/phase-2/prob-2/processed/'
train_data = '../../data/train_data/phase-2/prob-2/train_x.parquet'
valid_data = '../../data/train_data/phase-2/prob-2/test_x.parquet'
raw_data = '../../data/raw_data/phase-2/prob-2/raw_train.parquet'
y_train_data = '../../data/train_data/phase-2/prob-2/train_y.parquet'
y_valid_data = '../../data/train_data/phase-2/prob-2/test_y.parquet'

In [7]:
train_x = pd.read_parquet(train_data)
valid_x = pd.read_parquet(valid_data)
feature_columns = train_x.columns.tolist()
train_x["label"] = pd.read_parquet(y_train_data)["label"]
valid_x["label"] = pd.read_parquet(y_valid_data)["label"]

In [8]:
total_x = pd.concat([train_x, valid_x])

In [9]:
dup_x = total_x.groupby([*feature_columns, *['label']]).agg(count_per_label = ('label', 'count')).reset_index()
dup_x['order'] = dup_x.sort_values(['count_per_label', 'label'], ascending=[False, True]).groupby(feature_columns).cumcount()

In [54]:
prediction = np.array(["Normal" for i in range(total_x.shape[0])])

In [8]:
dict_dup = {}
captured_x = pd.DataFrame()
count_ = 0
for file_path in os.listdir(link_data_test):
    if 'parquet' not in file_path or "123.parquet" in file_path:
        continue
    captured_data = pd.read_parquet(f'{link_data_test}{file_path}')
    count_dup = captured_data[feature_columns].groupby(feature_columns).agg(count_unique = ('feature1', 'count'))
    count_dup = count_dup[count_dup['count_unique'] > 1].shape[0]
    captured_data['parquet_idx'] = count_
    if count_dup not in dict_dup:
        dict_dup[count_dup] = 0
    dict_dup[count_dup] += 1
    captured_x = pd.concat([captured_x, captured_data])
    count_ += 1

In [11]:
dict_dup

{23: 7,
 51: 7,
 58: 9,
 46: 15,
 205: 2,
 42: 6,
 45: 7,
 111: 2,
 59: 10,
 39: 6,
 49: 5,
 31: 5,
 48: 5,
 28: 9,
 47: 2,
 44: 5,
 25: 5,
 24: 8,
 38: 2,
 83: 2,
 41: 2,
 192: 2,
 32: 4,
 211: 1,
 22: 5,
 207: 4,
 318: 3,
 27: 7,
 52: 4,
 34: 5,
 20: 2,
 40: 2,
 37: 1,
 26: 3,
 29: 5,
 319: 2,
 54: 1,
 30: 5,
 84: 3,
 35: 1,
 53: 1,
 197: 1,
 115: 1,
 33: 4,
 21: 1,
 123: 1,
 317: 1,
 204: 1,
 316: 1,
 56: 1,
 116: 1,
 50: 1,
 124: 1,
 133: 1,
 82: 1,
 215: 1,
 198: 1}

In [24]:
captured_x[captured_x['drift'] == 1]['parquet_idx'].shape

(15568,)

In [27]:
test_unique_drift = captured_x[captured_x['drift'] == 1].groupby(feature_columns).agg(parquet_unique = ('parquet_idx', 'nunique')).reset_index()

In [31]:
test_unique_drift[test_unique_drift['parquet_unique'] != 1]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,parquet_unique
0,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,14
1,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,7
2,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,14
3,0.000000,-1,0,-1,1.0,0.0,46.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,7
4,0.000000,-1,0,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,58.881458,78,-1,5,68.0,0.0,18496.0,0.0,254.0,0.0,...,2.0,2.0,5.0,0.0,0.0,0.0,4.0,2.0,0.0,14
1174,59.469406,78,0,-1,80.0,0.0,8640.0,0.0,254.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,7
1175,59.810604,-1,0,4,130.0,0.0,35360.0,0.0,254.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,14
1176,59.998463,-1,0,-1,2.0,0.0,92.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,13


In [15]:
test_drift = captured_x.groupby(feature_columns).agg(count_record = ('feature1', 'count'), parquet_unique = ('parquet_idx', 'nunique'), drift_unique = ('drift', 'nunique')).reset_index()

In [18]:
test_drift[test_drift["drift_unique"] == 2]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature35,feature36,feature37,feature38,feature39,feature40,feature41,count_record,parquet_unique,drift_unique
46,0.000000,6,0,4,1.0,0.0,46.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,26,19,2
580,0.000003,118,2,4,2.0,0.0,114.0,0.0,254.0,0.0,...,25.0,0.0,0.0,0.0,15.0,25.0,0.0,25,25,2
799,0.000004,96,0,4,2.0,0.0,640.0,0.0,254.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,0.0,27,20,2
804,0.000004,115,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,13.0,0.0,0.0,0.0,6.0,13.0,0.0,26,26,2
935,0.000004,118,2,4,2.0,0.0,114.0,0.0,254.0,0.0,...,47.0,0.0,0.0,0.0,24.0,47.0,0.0,27,20,2
955,0.000004,119,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,5.0,0.0,0.0,0.0,4.0,5.0,0.0,27,27,2
967,0.000004,119,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,2.0,0.0,0.0,0.0,4.0,14.0,0.0,26,26,2
968,0.000004,119,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,5.0,0.0,0.0,0.0,9.0,14.0,0.0,27,27,2
1168,0.000005,118,2,4,2.0,0.0,114.0,0.0,254.0,0.0,...,30.0,0.0,0.0,0.0,13.0,30.0,0.0,26,26,2
1182,0.000005,118,2,4,2.0,0.0,114.0,0.0,254.0,0.0,...,33.0,0.0,0.0,0.0,34.0,33.0,0.0,26,26,2


In [33]:
train_dup_test = train_x.merge(test_idx_train_dup, how='inner', on=feature_columns)
train_dup_test

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label,idx,unique_label
0,0.000001,27,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,5.0,0.0,0.0,0.0,4.0,5.0,0.0,Denial of Service,271,4
1,0.000001,27,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,5.0,0.0,0.0,0.0,4.0,5.0,0.0,Denial of Service,26401,4
2,0.000001,27,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,5.0,0.0,0.0,0.0,4.0,5.0,0.0,Exploits,26624,4
3,0.000001,27,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,5.0,0.0,0.0,0.0,4.0,5.0,0.0,Information Gathering,30387,4
4,0.000001,27,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,5.0,0.0,0.0,0.0,4.0,5.0,0.0,Malware,32156,4


In [14]:
test_drift

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature35,feature36,feature37,feature38,feature39,feature40,feature41,count_record,parquet_unique,drift_unique
0,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,1.0,14,14,1
1,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,1.0,14,7,1
2,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,1.0,14,14,1
3,0.000000,-1,0,-1,1.0,0.0,46.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,14,7,1
4,0.000000,-1,0,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,1.0,14,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7959,59.996918,6,0,4,2.0,0.0,92.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,13,13,1
7960,59.998463,-1,0,-1,2.0,0.0,92.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,1.0,13,13,1
7961,59.998463,-1,0,-1,2.0,0.0,92.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,1.0,1,1,1
7962,59.999935,6,-1,-1,2.0,0.0,92.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,1.0,14,7,1


In [16]:
test_drift[test_drift['drift_unique'] == 2]['count_record'].sum()

1521

In [21]:
feature_group = train_x.columns.tolist()
feature_group.remove("label")
dup_train = train_x.groupby(feature_group).agg(label_nunique=("label", "nunique")).reset_index()

In [23]:
dup_train[dup_train["label_nunique"] != 1]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label_nunique
296,0.000000,119,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,3.0,3.0,9.0,0.0,0.0,0.0,5.0,7.0,0.0,2
297,0.000001,0,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,1.0,1.0,5.0,0.0,0.0,0.0,2.0,5.0,0.0,3
298,0.000001,21,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,4.0,4.0,8.0,0.0,0.0,0.0,5.0,5.0,0.0,3
299,0.000001,27,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,3.0,3.0,5.0,0.0,0.0,0.0,4.0,5.0,0.0,4
300,0.000001,29,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,1.0,1.0,4.0,0.0,0.0,0.0,9.0,4.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35634,59.687565,78,0,4,106.0,0.0,28832.0,0.0,254.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3
35638,59.744881,78,0,5,68.0,0.0,18496.0,0.0,254.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3
35639,59.760056,78,0,4,180.0,0.0,48960.0,0.0,254.0,0.0,...,1.0,1.0,4.0,0.0,0.0,0.0,24.0,1.0,0.0,3
35642,59.810604,78,0,4,130.0,0.0,35360.0,0.0,254.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,2


In [24]:
dup_train["feature2"].unique()

array([  6, 118, 119,   0,  21,  27,  29,  30,  32,  35,  38,  44,  46,
        49,  53,  56,  58,  66,  67,  69,  72,  75,  77,  89,  90,  92,
        95,  96,  99, 103, 109, 112, 116, 122, 123, 126, 128,   1,   3,
         9,  15,  25,  34,  52,  60,  70,  76,  84,  93, 108, 117, 124,
       125, 130,   4,   8,  10,  12,  13,  14,  17,  18,  19,  22,  23,
        24,  31,  33,  36,  39,  40,  41,  42,  43,  45,  47,  50,  51,
        54,  57,  59,  61,  62,  63,  64,  65,  68,  79,  80,  82,  83,
        85,  86,  87,  88,  94,  98, 100, 101, 102, 110, 111, 113, 114,
       127, 129, 131,   2,   5,   7,  26,  71,  81,  97, 105, 106, 115,
        20,  48,  55, 104, 121,  16,  78, 120,  74,  28,  37, 107,  11,
        73,  91])

In [25]:
dup_test = captured_x.groupby(feature_group).agg(count_num=("feature1", "count")).reset_index()

In [26]:
dup_test[dup_test["feature3"] == -1]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,count_num
0,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,42
1,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,42
2,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,42
9,0.000000,6,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,42
10,0.000000,6,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8982,33.520821,78,-1,-1,70.0,0.0,7560.0,0.0,254.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,3.0,6.0,0.0,42
9012,47.485291,78,-1,5,70.0,0.0,19040.0,0.0,254.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,6.0,2.0,0.0,42
9022,51.071350,112,-1,-1,20.0,14.0,1000.0,622.0,254.0,252.0,...,3.0,3.0,5.0,0.0,0.0,0.0,4.0,6.0,0.0,42
9046,58.881458,78,-1,5,68.0,0.0,18496.0,0.0,254.0,0.0,...,2.0,2.0,5.0,0.0,0.0,0.0,4.0,2.0,0.0,42


In [27]:
merge_ = dup_train.merge(dup_test, how="inner", on=feature_group)

In [32]:
merge_[merge_["label_nunique"] != 1]["count_num"].sum()

33152